In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings

In [3]:
print(os.listdir())


['.config', 'sample_data']


In [4]:
os.chdir("../")
%pwd

'/'

In [5]:
train_data=pd.read_csv('preprocessed_train_data.csv')
test_data=pd.read_csv('preprocessed_test_data.csv')

In [6]:
train_x=train_data.drop(['Item_Outlet_Sales'],axis=1)
train_y=train_data['Item_Outlet_Sales']
test_x=test_data.drop(['Item_Outlet_Sales'],axis=1)
test_y=test_data['Item_Outlet_Sales']

In [10]:
# Initialize RandomForestRegressor
rf = RandomForestRegressor()

# Define hyperparameters to tune
param_grid = {
    'n_estimators': [50, 100, 200],  # Number of trees in the forest
    'max_depth': [None, 10, 20, 30],  # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],  # Minimum samples required to split an internal node
    'min_samples_leaf': [1, 2, 4],  # Minimum samples required at each leaf node
    'max_features': ['auto', 'sqrt', 'log2'],  # Number of features to consider for best split
    'bootstrap': [True, False]  # Whether bootstrap samples are used when building trees
}

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, verbose=2, n_jobs=-1, scoring='neg_mean_squared_error')

# Fit grid search
grid_search.fit(train_x, train_y)

# Get the best parameters and the best model
print("Best Hyperparameters:", grid_search.best_params_)
best_rf = grid_search.best_estimator_

# Evaluate on test data
test_rmse = np.sqrt(-grid_search.score(test_x,test_y))  # RMSE from neg_mean_squared_error scoring
print(f'Test RMSE: {test_rmse}')


Fitting 5 folds for each of 648 candidates, totalling 3240 fits


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
1080 fits failed out of a total of 3240.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
405 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py

Best Hyperparameters: {'bootstrap': True, 'max_depth': 30, 'max_features': 'log2', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 100}
Test RMSE: 1049.9310562260557


In [7]:
from sklearn.model_selection import  cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LinearRegression, RidgeCV
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error,r2_score
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import AdaBoostRegressor

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [10]:
# Define the GradientBoostingRegressor model
gbr = GradientBoostingRegressor()

# Define the hyperparameter grid to tune
param_grid = {
    'n_estimators': [50, 100, 150, 200, 250],
    'learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3],
    'max_depth': [3, 5, 7, 9, 11],
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 5, 10],
    'subsample': [0.7, 0.8, 0.9, 1.0],
    'max_features': ['auto', 'sqrt', 'log2', None],
    'loss': ['ls', 'huber', 'absolute_error']
}
# Set up randomSearchCV
random_search = RandomizedSearchCV(estimator=gbr, param_distributions=param_grid,n_iter=100, cv=5, verbose=2, n_jobs=-1, scoring='neg_mean_squared_error')

# Fit random search
random_search.fit(train_x, train_y)

# Get the best parameters and the best model
print("Best Hyperparameters:", random_search.best_params_)
best_rf = random_search.best_estimator_

# Evaluate on test data
test_rmse = np.sqrt(-random_search.score(test_x,test_y))  # RMSE from neg_mean_squared_error scoring
print(f'Test RMSE: {test_rmse}')

Fitting 5 folds for each of 100 candidates, totalling 500 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
260 fits failed out of a total of 500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
153 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1466, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/util

Best Hyperparameters: {'subsample': 0.8, 'n_estimators': 250, 'min_samples_split': 10, 'min_samples_leaf': 5, 'max_features': None, 'max_depth': 3, 'loss': 'absolute_error', 'learning_rate': 0.1}
Test RMSE: 1027.9806413263875


In [12]:


# Initialize GradientBoostingRegressor with the specified hyperparameters
gbr = GradientBoostingRegressor(
    subsample=0.8,              # Fraction of samples to be used for fitting each base learner
    n_estimators=250,           # Number of boosting stages to be used
    min_samples_split=10,       # Minimum number of samples required to split an internal node
    min_samples_leaf=5,         # Minimum number of samples required to be at a leaf node
    max_features=None,          # Number of features to consider for the best split
    max_depth=3,                # Maximum depth of the individual trees
    loss='absolute_error',      # Loss function to optimize
    learning_rate=0.1          # Step size shrinking to improve model generalization
)

# Fit the model on the training data
gbr.fit(train_x, train_y)

# Predict on the training data
predict_value_train = gbr.predict(train_x)

# Calculate RMSE and R² on the training data
train_rmse = np.sqrt(mean_squared_error(train_y, predict_value_train))
train_r2 = r2_score(train_y, predict_value_train)

# Output training results
print(f'Train RMSE: {train_rmse}')
print(f'Train R²: {train_r2}')

# Predict on the test data
test_predict = gbr.predict(test_x)

# Calculate RMSE and R² on the test data (assuming you have test_y as the true values)
test_rmse = np.sqrt(mean_squared_error(test_y, test_predict))
test_r2 = r2_score(test_y, test_predict)

# Output test results
print(f'Test RMSE: {test_rmse}')
print(f'Test R²: {test_r2}')


Train RMSE: 1057.9738759808943
Train R²: 0.6216059116660508
Test RMSE: 1029.1322550937346
Test R²: 0.6103293367786498
